# Coursera - Analyzing criminal incident data 

## Install the RKernel

In [2]:
install.packages(c('rzmq','repr','IRkernel','IRdisplay'),
                  repos = c('http://irkernel.github.io/',     
                  getOption('repos')),
                  type = 'source')

Installing packages into ‘/Users/eweusthof/Library/R/3.2/library’
(as ‘lib’ is unspecified)
Warning message:
In install.packages(c("rzmq", "repr", "IRkernel", "IRdisplay"), : installation of package ‘rzmq’ had non-zero exit statusWarning message:
In install.packages(c("rzmq", "repr", "IRkernel", "IRdisplay"), : installation of package ‘IRkernel’ had non-zero exit status


The downloaded source packages are in
	‘/private/var/folders/mj/1pd8v_xx3gz7vlkz3d90xypr0000gn/T/RtmpxuFqf4/downloaded_packages’


In [1]:
IRkernel::installspec()


## What type of question can be answered? 

In [ ]:
#For either city, how do incidents vary by time of day? Which incidents are most common in the evening? 
# During what periods of the day are robberies most common?

#For either city, how do incidents vary by neighborhood? Which incidents are most common in the city center? 
# In what areas or neighborhoods are robberies or thefts most common?

#For either city, how do incidents vary month to month in the Summer 2014 dataset?

#For either city, which incident types tend to correlate with each other on a day-by-day basis?

#Advanced What can we infer broadly about the differences in crime patterns between Seattle and San Francisco? 
#Does one city tend to have more crime than the other, per capita? 
# Do the relative frequencies of types of incidents change materially between the two cities? (NOTE: The two datasets do not have the same schema, so comparisons will require some work and some assumptions. This will require extra work, but you will be working at the forefront of what is known!)

#Advanced For either city, do certain crimes correlate with environmental factors such as temperature? (To answer this kind of question, you will need to identify and use external data sources!)

## Read the Data

In [4]:
san.data <- read.csv("assignment6/sanfrancisco_incidents_summer_2014.csv")

In [7]:
seat.data <- read.csv("assignment6/seattle_incidents_summer_2014.csv")

In [8]:
head(seat.data)

,RMS.CDW.ID,General.Offense.Number,Offense.Code,Offense.Code.Extension,Offense.Type,Summary.Offense.Code,Summarized.Offense.Description,Date.Reported,Occurred.Date.or.Date.Range.Start,Occurred.Date.Range.End,Hundred.Block.Location,District.Sector,Zone.Beat,Census.Tract.2000,Longitude,Latitude,Location,Month,Year
1,483839,2015218538,2202,0,BURGLARY-FORCE-RES,2200,BURGLARY,06/28/2015 10:31:00 AM,06/28/2014 10:31:00 AM,06/28/2015 10:31:00 AM,6XX BLOCK OF NW 74 ST,J,J2,2900.301,-122.3647,47.68252,"(47.68252427, -122.364671996)",6,2014
2,481252,2015213067,2610,0,FRAUD-IDENTITY THEFT,2600,FRAUD,06/24/2015 11:09:00 AM,06/01/2014 12:00:00 AM,06/24/2015 11:09:00 AM,23XX BLOCK OF 43 AV E,C,C2,6300.1,-122.2771,47.6399,"(47.639900761, -122.277080248)",6,2014
3,481375,2015210301,2316,0,THEFT-MAIL,2300,MAIL THEFT,06/22/2015 09:22:00 AM,08/31/2014 09:00:00 AM,,81XX BLOCK OF 11 AV SW,F,F3,11300.5,-122.3493,47.52923,"(47.529232299, -122.349312181)",8,2014
4,481690,2015209327,2599,0,COUNTERFEIT,2500,COUNTERFEIT,06/21/2015 03:52:00 PM,06/20/2014 01:38:00 PM,,6XX BLOCK OF PINE ST,M,M2,8200.1,-122.3348,47.61237,"(47.612368448, -122.334817763)",6,2014
5,478198,2015207880,2399,3,THEFT-OTH,2300,OTHER PROPERTY,06/20/2015 11:59:00 AM,06/01/2014 11:59:00 AM,11/01/2014 12:00:00 PM,77XX BLOCK OF SUNNYSIDE AV N,J,J3,2700.202,-122.3294,47.68596,"(47.685959879, -122.329378505)",6,2014
6,480485,2015904103,2308,0,THEFT-BUILDING,2300,OTHER PROPERTY,06/19/2015 02:55:00 PM,06/19/2014 02:45:00 PM,07/10/2014 02:45:00 PM,35XX BLOCK OF S FERDINAND ST,R,R3,10300.4,-122.2875,47.55785,"(47.557854802, -122.287477902)",6,2014


In [9]:
head(san.data)

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
1,140734311,ARSON,ARSON OF A VEHICLE,Sunday,08/31/2014,23:50,BAYVIEW,NONE,LOOMIS ST / INDUSTRIAL ST,-122.4056,37.73832,"(37.7383221869053, -122.405646994567)",1.407343e+13
2,140736317,NON-CRIMINAL,LOST PROPERTY,Sunday,08/31/2014,23:45,MISSION,NONE,400 Block of CASTRO ST,-122.435,37.76177,"(37.7617677182954, -122.435012093789)",1.407363e+13
3,146177923,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,08/31/2014,23:30,SOUTHERN,NONE,1000 Block of MISSION ST,-122.4098,37.78004,"(37.7800356268394, -122.409795194505)",1.461779e+13
4,146177531,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,08/31/2014,23:30,RICHMOND,NONE,FULTON ST / 26TH AV,-122.4853,37.77252,"(37.7725176473142, -122.485262988324)",1.461775e+13
5,140734220,NON-CRIMINAL,FOUND PROPERTY,Sunday,08/31/2014,23:23,RICHMOND,NONE,800 Block of LA PLAYA ST,-122.5099,37.77231,"(37.7723131976814, -122.509895418239)",1.407342e+13
6,140734349,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Sunday,08/31/2014,23:13,SOUTHERN,"ARREST, BOOKED",11TH ST / MINNA ST,-122.4166,37.77391,"(37.773907074489, -122.416578493475)",1.407343e+13


In [ ]:
#For either city, how do incidents vary by time of day? Which incidents are most common in the evening? 
